# Find the 3 signals (ECG, ABP, PPG)
From the downloaded file, this function first fills in a 'signals' matrix which contains all the acquired signals and then separates these signals into 3 categories: AP, ECG and PPG signals.

Input : path to your file (needs to be already downloaded and saved in the same place as the code)

Outputs : 3 signals for the artefacts detection (ECG, ABP, PPG), tm is the scale time corresponding to the signals, sampling frequency fs 

In [2]:
import wfdb
import numpy as np

def find_signal(path):
    signals, fields = wfdb.rdsamp(path) #use a wfdb function in the wfdb package made by PhysioNet (website from which I downloaded the files)
    #takes a path and return the values of the signals, for example if there is 4 signals the matrix will be X rows and 4 columns 
    #fields is the informations contained in the header file (sampling frequency, nbr signals, etc...)
    list=[]
    fs = fields['fs'] #to have the frequency of sampling 
    ts = 1/fs #sample time
    L = len(signals) #Length of signal
    tm = np.linspace(0,(L-1)*ts,L) #time vector
    ppg_signal=np.empty([len(signals),1])
    abp_signal=np.empty([len(signals),1])
    for k in range(signals.shape[1]): #loop on the columns of the 'signals' matrix 
        if fields['sig_name'][k]=='ABP': #take the blood pressure signal from the 'signals' matrix if there is one 
            abp_signal=signals[:,k]
        if fields['sig_name'][k]=='PLETH': #take the PPG signal from the matrix if there is one 
            ppg_signal=signals[:,k]
        if fields['units'][k]=='mV': #take every ECG signal (there can be as many ECG signals as there are channels = 'V','II','I'... from 'I' to 'VI')  
            list.append(k) #if there an ECG signal I add the index in a list
            ecg_signal=np.empty([len(ppg_signal),len(list)])
        for i in range(len(list)): 
            k=list[i]
            ecg_signal=np.empty([len(signals),len(list)])
            ecg_signal[:,i]=signals[:,k] #fill a matrix with as many columns as there are ecg signals
    return abp_signal, ppg_signal, ecg_signal,tm, fs